# Nii图像的预处理与格式转化 

In [3]:
import nibabel as nib

nii_path = './brain1_image.nii.gz'
data_nii = nib.load(nii_path)
data1 = data_nii.get_fdata()
#self.shape = data1.shape
print(data1.shape) #(256,256,25) 表示各方向的切片数量
total_slices = data1.shape[2]

import shutil, os, nibabel
import imageio
import numpy as np

outputfile = 'result_png\\'       #输出文件夹
def niito2D(filepath):
    inputfiles = os.listdir(filepath)  #遍历文件夹数据
    print('Input file is ', inputfiles)
    print('Output folder is ', outputfile)

    for inputfile in inputfiles:
        image_array = nibabel.load(filepath+inputfile).get_fdata() #数据读取
        print(len(image_array.shape))

        # set destination folder
        if not os.path.exists(outputfile):
            os.makedirs(outputfile)   #不存在输出文件夹则新建
            print("Created ouput directory: " + outputfile)
        print('Reading NIfTI file...')

        mid_slice =  int(total_slices / 2)  #获取中间层，注意可能是浮点数
        slices = int(mid_slice / 2) #从第几个切片开始
        slice_counter = mid_slice #从第几个切片开始

        # iterate through slices
        #for current_slice in range(mid_slice-slices, mid_slice+slices):
        for current_slice in range(mid_slice, mid_slice+1):
            # alternate slices
            if (slice_counter % 1) == 0:
                data = image_array[:, :, current_slice]  #保存该切片
                data = data.astype(np.uint8)
                # alternate slices and save as png
                if (slice_counter % 1) == 0:
                    print('Saving image...')
                    #切片命名
                    image_name = inputfile[:-4] + "{:0>3}".format(str(current_slice + 1)) + ".png"
                    
                    #创建nii对应的图像的文件夹
                    img_f_path = os.path.join(outputfile, inputfile[:-6])
                    if not os.path.exists(img_f_path):
                        os.mkdir(img_f_path) #新建文件夹
                    
                    #保存
                    imageio.imwrite(image_name, data)
                    print('Saved.')

                    # move images to folder
                    print('Moving image...')
                    src = image_name
                    shutil.move(src, img_f_path)
                    slice_counter += 1
                    print('Moved.')

    print('Finished converting images')

niito2D('./Test_data/')


(256, 256, 25)
Input file is  ['brain1_image.nii.gz', 'brain1_label.nii.gz', 'MEN_001.nii', 'MEN_001.nii.gz', 'MEN_001_0000.nii', 'MEN_001_0000.nii.gz', 'MEN_001_0001.nii.gz', 'MEN_001_0002.nii.gz', 'MEN_001_0003.nii.gz']
Output folder is  result_png\
3
Created ouput directory: result_png\
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
3
Reading NIfTI file...
Saving image...
Saved.
Moving image...
Moved.
Finished converting images


## 路径准备

In [16]:
import torch
import scipy.io as io
def print_features(feat):
    #feafile = open('./features.txt', 'a')
    
    result1 = np.array(feat)
    np.savetxt('feature.txt',result1)
    #io.savemat('save.mat',{'result1':result1})
    #with open('features.txt', 'a') as f:
    #    x_flat = torch.flatten(feat)
    #    print(x_flat)
    #    for i in x_flat:
    #        f.write(str(i))
    #for feature in feat:
    #    print(*feature,file = feafile)

# VGG特征提取
## VGG19

In [6]:
import torchvision.models as models


def vgg19_extractor(nimg):
     # 获取vgg19原始模型, 输出图像维度是1000.
    vgg_model_1000 = models.vgg19(pretrained=True)
 
    # 下面三行代码功能是:得到修改后的vgg19模型.
    # 具体实现是: 去掉vgg19原始模型的第三部分classifier的最后一个全连接层, 
    # 用新的分类器替换原始vgg19的分类器，使输出维度是4096.
    vgg_model_4096 = models.vgg19(pretrained=True)
    new_classifier = torch.nn.Sequential(*list(vgg_model_4096.children())[-1][:6])
    vgg_model_4096.classifier = new_classifier

 
    # 使用vgg19得到图像特征.
    # 原始vgg19模型
    print(vgg_model_1000(nimg).data.shape)
    image_feature_1000 = vgg_model_1000(nimg).data[0]
    print_features(image_feature_1000)
    #print('features of vgg_model_1000: ', 10*image_feature_1000)
    # 修改后的vgg19模型
    image_feature_4096 = vgg_model_4096(nimg).data[0]
    #print('features of vgg_model_4096: ', image_feature_4096)


## VGG16

In [ ]:
def vgg16_extractor(nimg):
    # 获取vgg16原始模型
    vgg16_model = models.vgg16(pretrained=True)

    # 下面三行代码功能是:得到修改后的vgg19模型.
    # 具体实现是: 去掉vgg19原始模型的第三部分classifier的最后一个全连接层, 
    # 用新的分类器替换原始vgg19的分类器，使输出维度是4096.
    #vgg_model_vgg16 = models.vgg16(pretrained=True)
    #new_classifier = torch.nn.Sequential(*list(vgg_model_4096.children())[-1][:6])
    #vgg_model_4096.classifier = new_classifier

    image_feature_vgg16 = vgg16_model(nimg).data[0]
    print('features of vgg_model_vgg16: ', image_feature_vgg16)

# ResNet
## ResNet18

In [ ]:
def ResNet18_extractor(nimg):
    # 获取vgg16原始模型
    ResNet18_model = models.resnet18(pretrained=True)

    image_feature_ResNet18 = ResNet18_model(nimg).data[0]
    print('features of vgg_model_ResNet18: ', image_feature_ResNet18)

## ResNet3D

# Densenet121

In [1]:
def DenseNet121_extractor(nimg):
    # 获取vgg16原始模型
    DenseNet121_model = models.densenet121(pretrained=True)

    image_feature_DenseNet121 = DenseNet121_model(nimg).data[0]
    print('features of vgg_model_DenseNet: ', image_feature_DenseNet121)

# GoogleNet

In [ ]:
def GoogleNet_extractor(nimg):
    # 获取vgg16原始模型
    GoogleNet_model = models.googlenet(pretrained=True)

    # 下面三行代码功能是:得到修改后的vgg19模型.
    # 具体实现是: 去掉vgg19原始模型的第三部分classifier的最后一个全连接层, 
    # 用新的分类器替换原始vgg19的分类器，使输出维度是4096.
    #vgg_model_vgg16 = models.vgg16(pretrained=True)
    #new_classifier = torch.nn.Sequential(*list(vgg_model_4096.children())[-1][:6])
    #vgg_model_4096.classifier = new_classifier

    image_feature_GoogleNet = GoogleNet_model(nimg).data[0]
    print('features of vgg_model_GoogleNet: ', image_feature_GoogleNet)

# 运行

In [18]:
import os
import glob
from torchvision import transforms
from PIL import Image

all_image_dir = outputfile
if (os.path.exists(all_image_dir)):
    #获取该目录下的所有文件或文件夹目录路径
        files = glob.glob(all_image_dir + '\\*' )
        #print(files)

# 获取和处理图像
for file in files:
      print(file) #各个nii图像切片的文件夹
      image_dir = os.walk(file)
      print(image_dir) #对文件夹下的每个图像进行遍历
      for path, dir_lst, file_lst in image_dir:
            for file_name in file_lst:
                image = os.path.join(path, file_name)
                print(image) #此处为图片名称
                for i in range(1,2):
                    im = Image.open(image).convert('RGB')
                #img = img.astype(np.uint8)
                    trans_vgg = transforms.Compose([
    
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])
                    
                    nimg = trans_vgg(im)
                    nimg.unsqueeze_(dim=0)

                    #调用特征提取网络
                    vgg19_extractor(nimg)
                    #vgg16_extractor(nimg)

                    #对于resnet
                    trans_res = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor()]
                    )
                    nimg_res = trans_res(im)
                    nimg_res.unsqueeze_(dim=0)

                    #调用特征提取网络
                    #ResNet18_extractor(nimg_res)
                    #vgg16_extractor(nimg)


result_png\brain1_image
<generator object _walk at 0x000001280BA14660>
result_png\brain1_image\brain1_image.ni013.png
torch.Size([1, 1000])
tensor([-1.0146e+00, -1.8942e+00,  4.6389e+00,  4.2632e+00,  3.9060e+00,
         8.2078e+00,  6.7256e+00, -3.7284e+00, -2.1842e+00, -8.8409e-01,
        -4.4203e+00, -4.5424e+00, -2.0048e+00,  5.7093e-02, -3.2038e+00,
        -2.5380e+00, -6.6733e+00, -2.7076e+00, -1.0671e+00, -9.9650e-01,
        -1.7816e+00, -5.9670e+00, -5.0088e+00, -4.1403e+00, -9.2969e-01,
        -8.3644e-01,  1.5534e+00, -9.2032e-01,  3.1218e+00, -1.0476e+00,
         2.5319e+00,  2.2245e+00,  3.2978e+00,  5.3785e+00,  8.4733e+00,
         3.4494e+00,  4.5192e+00, -2.3908e-01,  2.3936e+00, -1.5399e-01,
        -8.9084e-01, -2.6149e+00, -3.6349e+00, -1.9360e-01, -3.5883e-01,
         1.1071e-01, -4.0040e+00, -1.4871e+00,  6.1726e-01,  1.1886e+00,
         3.8010e+00,  5.0333e+00, -5.0999e-01,  4.2212e-02,  4.5615e-01,
        -1.7321e+00, -4.1598e+00, -1.6678e+00, -8.3345e-0